# Metric Calculation for Extractive Question Answering tasks

Here the F1, SEMSCORE Calculation for extractive Question Answering is done. it can be replicated for cloze filling task in same fashion.

**F1 scores are calculated using 2 different methods.**


In method 1 every string is tokenize into words and these words are compared to the response text.
In method 2 every string is tokenize into alphabets and symbols (1-gram) and these symbols are compared to the response text.

In [ ]:
#Checking if GPU is available for the notebook or not
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
cuda_available = torch.cuda.is_available()

if cuda_available:
    print("CUDA is available.")
else:
    print("CUDA is not available.")

CUDA is not available.


**Methods to calculate F1 score given target text and response text**

In [ ]:
import collections

#Function to get words as a list
def get_token(text):
    # Split the text into words using whitespace
    tokens = str(text).split()
    return tokens


#Function to get F1 score by method1 (every string is tokenize into words and these words are compared to the response text.)
def compute_f1_method1(a_gold,a_pred):
  gold_toks=get_token(a_gold) #target tokens
  pred_toks=get_token(a_pred) #predicted tokens
  common= collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same=sum(common.values())

  if len(gold_toks)==0 or len(pred_toks)==0:
    return int(gold_toks==pred_toks)
  if num_same==0:
    return 0

  precision=1.0*num_same/len(pred_toks)
  recall =1.0*num_same/len(gold_toks)

  f1=(2*precision*recall)/(precision +recall)
  return f1


#Function to get F1 score by method 2 ( every string is tokenize into alphabets and symbols (1-gram) and these symbols are compared to the response text. )
def compute_f1_method2(gold_toks, pred_toks):
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())

    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)

    f1 = (2 * precision * recall) / (precision + recall)
    return f1

**Functions for Calculating Cosine similiarity between target response and predicted response using sentence transforme all-mpnet-base-v2**

In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
#Creating instance for sentence-transformer 'all-mpnet-base-v2'
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('all-mpnet-base-v2')

#Function to calculate the SEMSCORE or cosine similarity of text embeddings from sentence transformers
def compute_SEMSCORE(model_response, target_response):
    model_embedding = model.encode(model_response)
    target_embedding = model.encode(target_response)

    # Compute cosine similarity
    similarity = cosine_similarity([model_embedding], [target_embedding])[0][0]

    return similarity

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Importing The generated response file and visualizing the head of dataframe**

In [ ]:
#Importing pandas
import pandas as pd
import csv

In [ ]:

#Accessing reponse csv file for t5 and roberta
df_t5 = pd.read_csv("/content/squad_with_t5_2.csv")
print(df_t5.shape)
df_roberta =pd.read_csv("/content/squad_with_roberta_3.csv")
print(df_roberta.shape)

(49999, 4)
(49999, 4)


In [ ]:
#Concatenating the responses from two models and renaming, dropping redundant columns
df = pd.merge(df_t5, df_roberta, on='Question')
df = df.rename(columns={'Context_x': 'Context', 'Response_x': 'Response'})
df = df.drop(columns=(["Context_y","Response_y"]))

#dropping the NAN data points, resetting the index and inspecting the head of dataframe
df=df.dropna()
df = df.reset_index(drop=True)
print(df.shape)
df.head()

(34200, 5)


,Context,Question,Response,Response by T5,Response by Roberta
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,singing and dancing,singing and dancing
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,Dangerously in Love (2003),2003
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas","Houston, Texas",Texas
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,1990s,1990s
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what R&B group was she the lead singer?,Destiny's Child,Destiny's Child,Destiny's Child


**Generating Reponse for all 34200 data points, Ecaluating F1 and SEMSCORE and saving it as csv**

In [ ]:
for i in range(34200):
  print(i,end="-")

  new_entry = {
          'Context': df['Context'][i],
          'Question': df['Question'][i],
          'Response': df['Response'][i],
          'Response by T5': df['Response by T5'][i],
          'Response by Roberta': df['Response by Roberta'][i],
          'F1 T5 by method 1': compute_f1_method1(str(df['Response'][i]),str(df['Response by T5'][i])) ,
          'F1 T5 by method 2': compute_f1_method2(str(df['Response'][i]),str(df['Response by T5'][i])),
          'F1 Roberta by method 1': compute_f1_method1(str(df['Response'][i]),str(df['Response by Roberta'][i])) ,
          'F1 Roberta by method 2': compute_f1_method2(str(df['Response'][i]),str(df['Response by Roberta'][i])),
          'F1 Roberta-T5 by method 1': compute_f1_method1(str(df['Response by T5'][i]),str(df['Response by Roberta'][i])) ,
          'F1 Roberta-T5 by method 2': compute_f1_method2(str(df['Response by T5'][i]),str(df['Response by Roberta'][i])),

          'Semscore T5':compute_SEMSCORE(str(df['Response'][i]),str(df['Response by T5'][i])),
          'Semscore Roberta':compute_SEMSCORE(str(df['Response'][i]),str(df['Response by Roberta'][i])),
          'Semscore T5-Roberta':compute_SEMSCORE(str(df['Response by T5'][i]),str(df['Response by Roberta'][i]))
          }

  with open('cloze_F1_semscore_final.csv', 'a') as f:
    pd.DataFrame([new_entry]).to_csv(f, header=f.tell()==0, index=False)

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99-100-101-102-103-104-105-106-107-108-109-110-111-112-113-114-115-116-117-118-119-120-121-122-123-124-125-126-127-128-129-130-131-132-133-134-135-136-137-138-139-140-141-142-143-144-145-146-147-148-149-150-151-152-153-154-155-156-157-158-159-160-161-162-163-164-165-166-167-168-169-170-171-172-173-174-175-176-177-178-179-180-181-182-183-184-185-186-187-188-189-190-191-192-193-194-195-196-197-198-199-200-201-202-203-204-205-206-207-208-209-210-211-212-213-214-215-216-217-218-219-220-221-222-223-224-225-226-227-228-229-230-231-232-233-234-235-236-237-238-239-240-241-242-243-244-245-246-247-248-249-250-251-252-253-254-255-256-257-258-259-260-261-262-263-264-265-266-267-268-269-270-271-272-273-274-275-276-27